## UM-Bridge  Integration

In this section, UM-Bridge model server can be found in the following. It is used to solve the ODEs by considering the value of $\mu$ as an input.

In [ ]:
import nest_asyncio
nest_asyncio.apply()
import numpy as np
import math
from scipy.integrate import odeint 
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.special as special
from mpl_toolkits.mplot3d import axes3d
from numpy import exp,arange
from pylab import meshgrid,cm,imshow,contour,clabel,colorbar,axis,title,show
import numpy
import umbridge

class PDEModel(umbridge.Model):
    def __init__(self):
        super().__init__("forward")
        self.t = np.linspace(0, 10, 100)
        self.x0 = -0.8
        self.y0 = 0
        self.z0 = 0 
        self.v_x0 = 0
        self.v_y0 = -0.6276410653920693
        self.v_z0 = 0
        self.parameter_init= [self.x0, self.y0, self.z0, self.v_x0, self.v_y0, self.v_z0]

              
    def get_input_sizes(self, config):
        return [1]  # for the input µ
        
    def get_output_sizes(self, config):
        return [100, 100, 100] # for the output x, y, z and v_x, v_y, v_z
        
    def __call__(self, parameters, config):
        mu = parameters[0][0]
        result = odeint(self.spacecraft, self.parameter_init, self.t, args=(mu,))
        a = result[:, 0].tolist()  # x
        b = result[:, 1].tolist()  # y
        c = result[:, 2].tolist()  # z
        return [a, b, c]  

    
    def spacecraft( self, parameters, t, mu):
        x, y, z, v_x, v_y, v_z = parameters
        
        r1 = np.sqrt((x + mu)**2 + y**2 + z**2)
        r2 = np.sqrt((x-1 + mu)**2 + y**2 + z**2)

        a_x = 2 * v_y + x - (1 - mu)*(x + mu)/(r1)**3 - (mu)*(x-1 + mu)/(r2)**3
        a_y = -2 * v_x + y - (1 - mu)*(y)/(r1)**3 - (mu)*(y)/(r2)**3 
        a_z = -(1 - mu)*(z)/(r1)**3 - (mu)*(z)/(r2)**3
    
        return [v_x, v_y, v_z, a_x, a_y, a_z]
        
        
    def supports_evaluate(self):
        return True

pde_model = PDEModel()
umbridge.serve_models([pde_model], 4243)

======== Running on http://0.0.0.0:4243 ========
(Press CTRL+C to quit)
